### Libraries

In [3]:
import numpy as np
import gymnasium as gym
import random
import imageio
import os
import tqdm

#import pickle5 as pickle
from tqdm.notebook import tqdm

### Start

In [ ]:
env = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=False, render_mode="rgb_array")

print("_____OBSERVATION SPACE_____ \n")
print("Observation Space", env.observation_space)
print("Sample observation", env.observation_space.sample())

print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample()) 

_____OBSERVATION SPACE_____ 

Observation Space Discrete(16)
Sample observation 2

 _____ACTION SPACE_____ 

Action Space Shape 4
Action Space Sample 0


### Q-table

In [15]:
state_space = env.observation_space.n
print("There are ", state_space, " possible states")

action_space = env.action_space.n
print("There are ", action_space, " possible actions")

def initialize_q_table(state_space, action_space):
  Qtable = np.zeros((state_space, action_space))
  return Qtable

Qtable_frozenlake = initialize_q_table(state_space, action_space)

There are  16  possible states
There are  4  possible actions


### Greedy policy

In [16]:
def greedy_policy(Qtable, state):
    
    action = np.argmax(Qtable[state][:])

    return action